<a href="https://colab.research.google.com/github/inbarasuvs/inba12/blob/main/AI_for_Product_Leaders_Prototyping_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install all the requirements and supress pip installation
!pip install -q loguru openai gradio >/dev/null 2>&1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Core imports for OpenAI API interaction and UI
import openai
import os
from loguru import logger #For logging
from google.colab import userdata #for handing API Keys
import gradio as gr #for UI

In [ ]:
#Fetch and ensure OpenAI API Keys are there
api_key = userdata.get('OPENAI_API_KEY')
logger.info(f"OpenAI API key: {'LOADED ✓' if api_key.startswith('sk-') else 'NOT FOUND ✗'}")

2024-11-22 13:21:21.764 | INFO     | __main__:<cell line: 3>:3 - OpenAI API key: LOADED ✓


In [ ]:
#Define OpenAI client
client = openai.OpenAI(api_key=api_key)

In [ ]:
def get_completion(prompt, model="gpt-4o-mini"):
    """
    Get completion from OpenAI API.

    Args:
        prompt (str): Input text to send to OpenAI
        model (str): OpenAI model to use, defaults to GPT-4

    Returns:
        str: Generated completion text
    """
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2
    )
    final_response = response.choices[0].message.content
    logger.info(f"Mode response: {final_response}")

    return final_response

In [ ]:
def generate_prompt(your_role, candidate_role, your_review):
    delimiter = "####"

    prompt = f"""
    I’m {your_role}. You’re an expert at writing performance reviews. On my behalf, help answer the question for performance reviews below.

    {delimiter} Instructions {delimiter}:
    - Use the context below to understand my perspective of working with him
    - Keep the role of person I’m review {candidate_role} in mind when writing the review
    - Use simple language and keep it to the point
    - Strictly answer the question mentioned in “question for performance”

    {your_review}

    - Describe example(s) of the topics selected. What was the context? What actions did they take?
    - In your opinion, what impact did their actions have?
    - What recommendations do you have for their growth and development? Your feedback can be about any area of their work.


    {delimiter} Output in markdown format in following structure:{delimiter}
    - Q1: Describe example(s) of the topics selected. What was the context? What actions did they take?
    Your answer
    - Q2: In your opinion, what impact did their actions have?
    Your answer
    - Q3: What recommendations do you have for their growth and development? Your feedback can be about any area of their work.
    Your answer

    Answer: """
    return prompt

In [ ]:
def generate_review(your_role, candidate_role, your_review):
    """
    Generate a structured performance review using OpenAI.

    Args:
        your_role (str): Reviewer's role
        candidate_role (str): Candidate's role
        your_review (str): Initial review text

    Returns:
        str: Formatted performance review
    """
    prompt = generate_prompt(your_role, candidate_role, your_review)
    response = get_completion(prompt)
    return response

In [ ]:
# Create a Gradio interface using Blocks for more flexible layout
with gr.Blocks() as demo:
    # Add header with decorative separators
    gr.Markdown("============================================================")
    gr.Markdown("Write performance review in a minute")  # Title
    gr.Markdown("============================================================")

    # Input Fields
    your_role = gr.Textbox(
        label='your role',  # e.g., "Senior Manager", "Team Lead"
    )

    candidate_role = gr.Textbox(
        label='candidate role',  # e.g., "Software Engineer", "Product Manager"
    )

    your_review = gr.Textbox(
        label='Briefly descrive your experience of working with candididate...',
        # This is the main input where reviewer provides detailed feedback
        # Can include:
        # - Project details
        # - Responsibilities
        # - Achievements
        # - Behavioral observations
    )

    # Action button to trigger review generation
    write_review = gr.Button("Write Review")

    # Output area where the AI-generated review will appear
    answer = gr.Markdown(
        label='Review'  # Will display formatted review in markdown
    )

    # Connect the interface:
    # When button is clicked:
    # 1. Take inputs from all three textboxes
    # 2. Pass them to generate_review function
    # 3. Display result in answer markdown
    write_review.click(
        fn=generate_review,            # Function to call
        inputs=[                       # List of inputs to pass to function
            your_role,
            candidate_role,
            your_review
        ],
        outputs=answer                 # Where to display the result
    )

# Launch the interface
demo.launch(
    share=True,     # Create a public link
    debug=True      # Show debugging information
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6b6bee9fc9af3e0d8d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2024-11-22 13:25:28.746 | INFO     | __main__:get_completion:19 - Mode response: - Q1: Describe example(s) of the topics selected. What was the context? What actions did they take?  
Mohak played a crucial role in the batch operations project for the Cloud Storage team. He assisted in writing the Product Requirements Document (PRD) and participated in design discussions, ensuring that all aspects of the project were well thought out. Additionally, he was actively involved in customer meetings, where he addressed customer questions and concerns. His contributions were vital in keeping the project on track and aligned with customer needs.

- Q2: In your opinion, what impact did their actions have?  
Mohak's actions had a significant positive impact on the project. His involvement in the PRD and design discussions helped clarify project goals and requirements, leading to a smoother development process. By engaging with customers directly, he ensured that their feedback was incorporated, w

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6b6bee9fc9af3e0d8d.gradio.live


In [ ]:
"""
Sample Review to test:

>I worked with Mohak on multiple projects. One of the projects was batch operations, which was a priority of Cloud Storage team, where he helped me write the PRD, do the design discussion. He was present in customer meetings and helped answer customer questions. Overall, this helped launch the product on time.

"""

'\nSample Review to test:\n\n>I worked with Mohak on multiple projects. One of the projects was batch operations, which was a priority of Cloud Storage team, where he helped me write the PRD, do the design discussion. He was present in customer meetings and helped answer customer questions. Overall, this helped launch the product on time.\n\n'